Dependencies

In [3]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, MaxPooling2D, Flatten, Conv2D
import tensorflow_datasets as tfds

Loading Dataset

In [4]:
train_dir="archive/train"
test_dir="archive/test"

# Function to load and preprocess images
def load_and_preprocess_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)  # Decode the image
    image = tf.image.resize(image, [64, 64])         # Resize the image
    image = tf.cast(image, tf.float32) / 255.0       # Normalize the image to [0, 1]
    return image, label

In [23]:
import tensorflow as tf
import os

# Define paths to your dataset
train_dir="archive/train"
test_dir="archive/test"

# Function to load and preprocess images
def load_and_preprocess_image(path, label):
    image = tf.io.read_file(path)  # Read the image file
    image = tf.image.decode_jpeg(image, channels=3)  # Decode the image to RGB
    image = tf.image.resize(image, [64, 64])         # Resize the image
    # Data augmentation
    image = tf.image.random_flip_left_right(image)   # Random horizontal flip
    image = tf.image.random_flip_up_down(image)      # Random vertical flip
    image = tf.image.random_contrast(image, lower=0.2, upper=0.5)  # Random contrast
    image = tf.cast(image, tf.float32) / 255.0 
    return image, label

# Function to create a dataset from directories
def create_dataset(directory, batch_size=32):
    # List image files and their corresponding labels (cat=0, dog=1)
    image_paths = []
    labels = []
    
    # Walk through the directories and collect paths and labels
    for label, class_name in enumerate(os.listdir(directory)):  # 0 for cat, 1 for dog
        class_dir = os.path.join(directory, class_name)
        if os.path.isdir(class_dir):
            for img_file in os.listdir(class_dir):
                image_paths.append(os.path.join(class_dir, img_file))
                labels.append(label)
    
    # Create a TensorFlow Dataset from the paths and labels
    path_ds = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    # Map the `load_and_preprocess_image` function to each image
    image_ds = path_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    # Shuffle, batch, and prefetch the dataset for better performance
    dataset = image_ds.shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    return dataset

# Create training and testing datasets
train_dataset = create_dataset(train_dir, batch_size=32)
test_dataset = create_dataset(test_dir, batch_size=32)

# Check the dataset output
for images, labels in train_dataset.take(1):
    print(images.shape, labels.shape)


(32, 64, 64, 3) (32,)


Data Augmentation


In [ ]:
def load_and_preprocess_image(path, label):
    image = tf.io.read_file(path)  # Read the image file
    image = tf.image.decode_jpeg(image, channels=3)  # Decode the image to RGB
    image = tf.image.resize(image, [64, 64])         # Resize the image
    
    # Data augmentation
    image = tf.image.random_flip_left_right(image)   # Random horizontal flip
    image = tf.image.random_flip_up_down(image)      # Random vertical flip
    image = tf.image.random_contrast(image, lower=0.2, upper=0.5)  # Random contrast
    
    image = tf.cast(image, tf.float32) / 255.0  # Normalize the image to [0, 1]
    return image, label

creating the model

In [24]:
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    # Conv2D(128, (3, 3), activation='relu'),
    # MaxPooling2D(2, 2),
    # Conv2D(64, (3, 3), activation='relu'),
    # MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer for binary classification (cat vs dog)
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [25]:
model.fit(
    train_dataset,
    epochs=25,
    validation_data=test_dataset,
    verbose=1
)

Epoch 1/25
18/18 ━━━━━━━━━━━━━━━━━━━━ 6s 205ms/step - accuracy: 0.4743 - loss: 0.9448 - val_accuracy: 0.5000 - val_loss: 0.6947
Epoch 2/25
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 165ms/step - accuracy: 0.5051 - loss: 0.6934 - val_accuracy: 0.5000 - val_loss: 0.6946
Epoch 3/25
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 170ms/step - accuracy: 0.4669 - loss: 0.6967 - val_accuracy: 0.4929 - val_loss: 0.6933
Epoch 4/25
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 155ms/step - accuracy: 0.5203 - loss: 0.6901 - val_accuracy: 0.4929 - val_loss: 0.7013
Epoch 5/25
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 144ms/step - accuracy: 0.5552 - loss: 0.6891 - val_accuracy: 0.5643 - val_loss: 0.6943
Epoch 6/25
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 145ms/step - accuracy: 0.5401 - loss: 0.6727 - val_accuracy: 0.5143 - val_loss: 0.7248
Epoch 7/25
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 147ms/step - accuracy: 0.5866 - loss: 0.6672 - val_accuracy: 0.5857 - val_loss: 0.6924
Epoch 8/25
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 148ms/step - accuracy: 0.5850 - loss: 0.6653 - val_accuracy: 0.

In [26]:
# Evaluate the model on the test dataset
test_loss, test_acc = model.evaluate(test_dataset)

# Print the results
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_acc * 100}")


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.5349 - loss: 0.7470
Test Loss: 0.7319509387016296
Test Accuracy: 55.000001192092896
